# 1) Loading and cleaning the data

In [1]:
import pandas as pd
import requests

In [2]:
df = pd.read_html('https://en.wikipedia.org/wiki/List_of_postal_codes_of_Canada:_M')
df = df[0]

#print('Before dropping Boroughs ='.join(df.shape))
df_wikipedia= df[df['Borough'] != 'Not assigned'].reset_index(drop= True)

#there are no Postal codes repeated so we dont need to concatinate the Neighbourhoods.

df_wikipedia.shape

(103, 3)

# 2) Loading LAtitude and logitude

In [3]:
!pip install geocoder
import geocoder

In [4]:
!wget -q -O 'zipcode.csv' http://cocl.us/Geospatial_data

In [5]:
#df.head(1)
#geocoder.google('M3A, Toronto, Ontario')

df1_zip = pd.read_csv('zipcode.csv')
df1_zip.head()

df_merged = pd.merge(df_wikipedia,df1_zip, on=['Postal Code'])

df_merged.head()

print(df_merged.size)

#geocoder.google('{}, Toronto, Ontario'.format(df['Postal Code'].iloc(0)))

515


In [6]:
df_merged.head()

,Postal Code,Borough,Neighborhood,Latitude,Longitude
0,M3A,North York,Parkwoods,43.753259,-79.329656
1,M4A,North York,Victoria Village,43.725882,-79.315572
2,M5A,Downtown Toronto,"Regent Park, Harbourfront",43.654260,-79.360636
3,M6A,North York,"Lawrence Manor, Lawrence Heights",43.718518,-79.464763
4,M7A,Downtown Toronto,"Queen's Park, Ontario Provincial Government",43.662301,-79.389494


# Filtering data frames to contain Boroughs that have the word Toronto in them

In [7]:
df_final=df_merged[df_merged['Borough'] .str.contains('Toronto')].reset_index(drop=True)

# These should yeild 39 locations.

df_final.head()
print(df_final.shape)

(39, 5)


# Now talk to Foursquare API to explore each of these locations and find categories of places around each within a radius of 10 M

In [8]:
import requests
from pandas.io.json import json_normalize


fourSqAPIURL = "https://api.foursquare.com/v2/venues/explore"
CLIENT_ID ="NINRXL4F12KA0T5Y40SQD1CGLLM2FK1XGE1YZBYMVQINX45V"
CLIENT_SECRET= "4NSTMXF2D5VWNSAMNRUMP3RC1W4NHOLDSVOLK3UX1BXBAZD5"

    

In [9]:
def fetch_venue_Details(postalcode,latitude,longitude,v_list):
    new_venue = {'Postal Code':postalcode,'Latitude':latitude, 'Longitude':longitude,'Type':[],'Name':[]}

    new_venue["Name"] = v_list["venue"]["name"]
    new_venue["Type"] = v_list["venue"]["categories"][0]['pluralName']
    

    new_venues = pd.DataFrame(new_venue, index =[0])
    
  
   # new_venues = new_venues.append(new_venue,ignore_index=True)
    #print('after--- '+ str(new_venues))
    return new_venues


In [11]:

all_venues= pd.DataFrame()


for ind in df_final.index:

   
    lat = str(df_final['Latitude'][ind])
    lng = str(df_final['Longitude'][ind])

    print(str(df_final['Postal Code'][ind])+ "+++++++++++"+ str(ind))
 
    payload = {'client_id': CLIENT_ID, 'client_secret': CLIENT_SECRET, 'll': lat+','+lng , 'radius':500, 'v':'20180323'}

    resp = requests.get(fourSqAPIURL,params=payload)

    data = resp.json()
    
   # print(data)
    try:
        for items in data["response"]["groups"][0]["items"]:
           
            all_venues = all_venues.append(fetch_venue_Details(df_final['Postal Code'][ind],lat,lng,items))
    except:
        print(str(df_final['Postal Code'][ind])+ "-----EMPTY------------")
        empty_venue = {'Postal Code':df_final['Postal Code'][ind],'Latitude':lat, 'Longitude':lng,'Type':[],'Name':[]}
        all_venues.append(pd.DataFrame(empty_venue, index =[0]))

M5A+++++++++++0
M7A+++++++++++1
M5B+++++++++++2
M5C+++++++++++3
M4E+++++++++++4
M5E+++++++++++5
M5G+++++++++++6
M6G+++++++++++7
M5H+++++++++++8
M6H+++++++++++9
M5J+++++++++++10
M6J+++++++++++11
M4K+++++++++++12
M5K+++++++++++13
M6K+++++++++++14
M4L+++++++++++15
M5L+++++++++++16
M4M+++++++++++17
M4N+++++++++++18
M5N+++++++++++19
M4P+++++++++++20
M5P+++++++++++21
M6P+++++++++++22
M4R+++++++++++23
M5R+++++++++++24
M6R+++++++++++25
M4S+++++++++++26
M5S+++++++++++27
M6S+++++++++++28
M4T+++++++++++29
M5T+++++++++++30
M4V+++++++++++31
M5V+++++++++++32
M4W+++++++++++33
M5W+++++++++++34
M4X+++++++++++35
M5X+++++++++++36
M4Y+++++++++++37
M7Y+++++++++++38


In [12]:


all_venues = all_venues.reset_index(drop=True)


all_venues.head()
#venues.append()


#for venue in data["response"]["groups"][0]["items"][0].items():
 #   venue

,Postal Code,Latitude,Longitude,Type,Name
0,M5A,43.6542599,-79.3606359,Bakeries,Roselle Desserts
1,M5A,43.6542599,-79.3606359,Coffee Shops,Tandem Coffee
2,M5A,43.6542599,-79.3606359,Breakfast Spots,Morning Glory Cafe
3,M5A,43.6542599,-79.3606359,Distribution Centers,Cooper Koo Family YMCA
4,M5A,43.6542599,-79.3606359,Spas,Body Blitz Spa East


In [13]:
neighbourhood_analysis = pd.get_dummies(all_venues,prefix='',prefix_sep='',columns=['Type'])

In [14]:
neighbourhood_analysis.shape

(861, 197)

In [15]:
mean_data = neighbourhood_analysis.set_index(['Postal Code'])
mean_data = mean_data.groupby(level='Postal Code').mean()

In [16]:
mean_data.head()



,Airport Food Courts,Airport Lounges,Airport Services,Airport Terminals,Airports,American Restaurants,Antique Shops,Aquariums,Art Galleries,Art Museums,...,Theme Restaurants,Toy / Game Stores,Trails,Train Stations,Vegetarian / Vegan Restaurants,Video Game Stores,Vietnamese Restaurants,Wine Bars,Wings Joints,Yoga Studios
Postal Code,,,,,,,,,,,,,,,,,,,,,
M4E,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,0.0,0.0,0.0,...,0.0,0.0,0.250000,0.0,0.0,0.0,0.0,0.000000,0.0,0.000000
M4K,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,0.0,0.0,0.0,...,0.0,0.0,0.033333,0.0,0.0,0.0,0.0,0.000000,0.0,0.033333
M4L,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,0.0,0.0,0.0,...,0.0,0.0,0.000000,0.0,0.0,0.0,0.0,0.000000,0.0,0.000000
M4M,0.0,0.0,0.0,0.0,0.0,0.033333,0.0,0.0,0.0,0.0,...,0.0,0.0,0.000000,0.0,0.0,0.0,0.0,0.033333,0.0,0.033333
M4N,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,0.0,0.0,0.0,...,0.0,0.0,0.000000,0.0,0.0,0.0,0.0,0.000000,0.0,0.000000


In [17]:
from sklearn.cluster import KMeans

#clusters
k=5


In [18]:
kmean = KMeans(n_clusters = k, random_state = 0).fit(mean_data.to_numpy())

In [19]:
kmean.labels_

array([0, 0, 0, 0, 3, 0, 0, 0, 2, 0, 2, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
       1, 4, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0], dtype=int32)

In [20]:
mean_data['Segment'] = kmean.labels_

In [21]:
#mean_data=mean_data.reset_index()

mean_data.groupby('Postal Code')


#mean_data[mean_data['Postal Code']=='M4E']

# Plotting the neighbourhoods in colors based on thier segment

In [22]:
df_merged = pd.merge(df_final,mean_data['Segment'], how='left', on='Postal Code')



In [23]:
df_merged

,Postal Code,Borough,Neighborhood,Latitude,Longitude,Segment
0,M5A,Downtown Toronto,"Regent Park, Harbourfront",43.654260,-79.360636,0
1,M7A,Downtown Toronto,"Queen's Park, Ontario Provincial Government",43.662301,-79.389494,0
2,M5B,Downtown Toronto,"Garden District, Ryerson",43.657162,-79.378937,0
3,M5C,Downtown Toronto,St. James Town,43.651494,-79.375418,0
4,M4E,East Toronto,The Beaches,43.676357,-79.293031,0
5,M5E,Downtown Toronto,Berczy Park,43.644771,-79.373306,0
6,M5G,Downtown Toronto,Central Bay Street,43.657952,-79.387383,0
7,M6G,Downtown Toronto,Christie,43.669542,-79.422564,0
8,M5H,Downtown Toronto,"Richmond, Adelaide, King",43.650571,-79.384568,0
9,M6H,West Toronto,"Dufferin, Dovercourt Village",43.669005,-79.442259,0


In [24]:
!pip install folium

     |████████████████████████████████| 102kB 6.9MB/s ta 0:00:011


In [77]:
import folium
 

m = folium.Map(location=[43.651070,-79.347015],zoom_start=12)

for ind in df_merged.index:

    div = folium.DivIcon(html=(
        '<svg height="50" width="50">'
        '<circle cx="15" cy="15" r="10" stroke="yellow" stroke-width="3" fill="yellow" />'
        '<text x="10" y="20" fill="black">'+str(df_merged['Segment'][ind])+ '</text>'
        '</svg>'
    ))
    folium.Marker((df_merged['Latitude'][ind],df_merged['Longitude'][ind]), icon=div).add_to(m)

m

#feature_group = folium.FeatureGroup("Locations")

#for lat, lng, name in zip(lat_lst, lng_lst, name_lst):
#feature_group.add_child(folium.Marker(location=[43.651070,-79.347015],popup='abcd'))


#m.add_child(feature_group)
#folium.map.Marker(popup='bc',radius=10,location=[43.651070,-79.347015]).add_to(m)